<a href="https://colab.research.google.com/github/janithcyapa/SmartHVAC/blob/main/SmartHVAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library
https://scikit-fuzzy.github.io/scikit-fuzzy/

In [ ]:
!pip install scikit-fuzzy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Simple HVAC

> Antecedents (inputs)
- Temperature: very cool, cool, moderate, high, very high (16 - 30 C)
- Humidity : Low, Medium, High (0 - 100)

> Consequents (outputs)
- Compressor fan speed : 0 - 100 rpm
- Cooler voltage out : 0 - 5 V
- Heater voltage : 0 - 5 V

> Rules

- IF Temp (High) AND RH (High) : THEN Compressor Fan Speed is Very High (e.g., ~90 rpm)

- IF Temp (Medium) AND RH (Low) : THEN Compressor Fan Speed is Medium (e.g., ~50 rpm)

- IF Temp (Low) AND RH (Medium) : THEN Compressor Fan Speed is Low (e.g., ~30 rpm)

In [ ]:
#inputs

temp = ctrl.Antecedent(np.arange(16,31,1),'temperature')
humidity = ctrl.Antecedent(np.arange(0,101,1),'humidity')
print(temp)
print(humidity)
print(temp.universe)
print(humidity.universe)

In [ ]:
#output

f_speed = ctrl.Consequent(np.arange(0,101,1),'f_speed')
c_volt = ctrl.Consequent(np.arange(0,6,1),'c_volt')
h_volt = ctrl.Consequent(np.arange(0,6,1),'h_volt')
print(f_speed)
print(c_volt)
print(h_volt)
print(f_speed.universe)
print(c_volt.universe)
print(h_volt.universe)

In [ ]:
#membership fn for inputs

temp.automf(number=5,names=['very cool', 'cool', 'moderate', 'high', 'very high'])
humidity.automf(number=3,names=['low', 'medium', 'high'])

temp.view()
humidity.view()

In [ ]:
#custom output membership fns for f_speed

f_speed['low']= fuzz.trimf(f_speed.universe,[0,0,50])
f_speed['medium']= fuzz.trimf(f_speed.universe,[0,50,100])
f_speed['high']= fuzz.trimf(f_speed.universe,[50,100,100])

f_speed.view()

#custom output membership fns for c_volt

c_volt['low']= fuzz.trimf(c_volt.universe,[0,0,3])
c_volt['medium']= fuzz.trimf(c_volt.universe,[0,3,5])
c_volt['high']= fuzz.trimf(c_volt.universe,[3,5,5])

c_volt.view()

#custom output membership fns for h_volt

h_volt['low']= fuzz.trimf(h_volt.universe,[0,0,3])
h_volt['medium']= fuzz.trimf(h_volt.universe,[0,3,5])
h_volt['high']= fuzz.trimf(h_volt.universe,[3,5,5])

h_volt.view()


In [ ]:
#rules for fuzzy logic


# Rule 1
rule1a = ctrl.Rule(temp['very high'] | humidity['high'], f_speed['high'])
rule1b = ctrl.Rule(temp['very high'] | humidity['high'], c_volt['high'])
rule1c = ctrl.Rule(temp['very high'] | humidity['high'], h_volt['high'])

# Rule 2
rule2a = ctrl.Rule(temp['high'] | humidity['medium'], f_speed['medium'])
rule2b = ctrl.Rule(temp['high'] | humidity['medium'], c_volt['medium'])
rule2c = ctrl.Rule(temp['high'] | humidity['medium'], h_volt['medium'])

# Rule 3
rule3a = ctrl.Rule(temp['moderate'] | humidity['low'], f_speed['low'])
rule3b = ctrl.Rule(temp['moderate'] | humidity['low'], c_volt['low'])
rule3c = ctrl.Rule(temp['moderate'] | humidity['low'], h_volt['low'])

# Rule 4
rule4a = ctrl.Rule(temp['cool'] | humidity['low'], f_speed['low'])
rule4b = ctrl.Rule(temp['cool'] | humidity['low'], c_volt['low'])
rule4c = ctrl.Rule(temp['cool'] | humidity['low'], h_volt['low'])

# Rule 5
rule5a = ctrl.Rule(temp['very cool'] | humidity['low'], f_speed['low'])
rule5b = ctrl.Rule(temp['very cool'] | humidity['low'], c_volt['low'])
rule5c = ctrl.Rule(temp['very cool'] | humidity['low'], h_volt['low'])


In [ ]:
#build control system
control_system = ctrl.ControlSystem([
    rule1a, rule1b, rule1c,
    rule2a, rule2b, rule2c,
    rule3a, rule3b, rule3c,
    rule4a, rule4b, rule4c,
    rule5a, rule5b, rule5c
])


In [ ]:
#build fuzzy system

fuzzy_system = ctrl.ControlSystemSimulation(control_system)
fuzzy_system


In [ ]:
#testing demo values

fuzzy_system.input['temperature'] = 25
fuzzy_system.input['humidity'] = 50

fuzzy_system.compute()

print("Fan Speed:", fuzzy_system.output['f_speed'])
print("Compressor Voltage:", fuzzy_system.output['c_volt'])
print("Heater Voltage:", fuzzy_system.output['h_volt'])

f_speed.view(sim = fuzzy_system)
c_volt.view(sim = fuzzy_system)
h_volt.view(sim = fuzzy_system)


# Connect LlaMA

In [ ]:
!pip install transformers accelerate bitsandbytes


In [ ]:
!pip install -U bitsandbytes accelerate transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define quantization config for 8-bit loading
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# Load model with quantization and automatic device placement
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)



In [ ]:
prompt = "Generate fuzzy logic rules for HVAC system where temperature and humidity inputs control fan speed and voltage outputs."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=150)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
from transformers import pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
generator = pipeline("text-generation", model=model_name, device=0)

prompt = """
You are an expert in control systems. Generate fuzzy logic control rules.

Inputs:
- Temperature: very cool, cool, moderate, high, very high
- Humidity: low, medium, high

Outputs:
- Fan Speed: low, medium, high
- Compressor Voltage: low, medium, high
- Heater Voltage: low, medium, high

Rules must follow this format excatly, only channge the [level] appropriately:
IF temperature is [level] AND/OR humidity is [level] THEN fan_speed is [level], compressor_voltage is [level], heater_voltage is [level]

Generate 5 rules:
"""

response = generator(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]['generated_text']
print(response)
